In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!pip install transformers

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
import logging
from huggingface_hub import notebook_login
from pathlib import Path

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

In [5]:
!pip install accelerate
!pip install auto-gptq
!pip install --upgrade optimum

In [7]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [8]:
def gen(p,maxlen=15, sample=True):
  toks = tokr(p , return_tensors='pt')
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to("cpu")
  return tokr.batch_decode(res)

In [9]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [10]:
def mk_pmt(user, syst=sb_sys): return f"{syst}###user: {user}\n\n### Assistant:\n"

In [11]:
ques = "Who is Jeremy Howard?"

In [6]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

## Retrival Augmented Generation


In [13]:
!pip install Wikipedia-api

In [14]:
from wikipediaapi import Wikipedia

In [15]:
wiki = Wikipedia("JeremyHowardBot/0.0",'en')
jh_pg = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_pg = jh_pg.split('\nReferences\n')[0]

In [12]:
print(jh_pg[:500])

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator.He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.

Early life
Howard was born in London, United Kingdom, and move


In [16]:
len(jh_pg.split())

613

In [17]:
ques_ctx = f"""Answer the question with the help of the provided context.

## Context

{jh_pg}

## Question

{ques}"""

In [18]:
res = gen(mk_pmt(ques_ctx),300)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


RuntimeError: ignored